# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
# Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
# sessionId = 338, and itemInSession = 4
drop_artist_song_query = "DROP TABLE IF EXISTS artist_song;"

create_artist_song_query = "CREATE TABLE IF NOT EXISTS artist_song "
create_artist_song_query = create_artist_song_query + "(artist_name text, song_name text, session_id int, item_in_session int, song_length float, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(drop_artist_song_query)
    session.execute(create_artist_song_query)
except Exception as e:
    print(e)
                    

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insert_artist_song_query = "INSERT INTO artist_song (artist_name, song_name, session_id, item_in_session, song_length)"
        insert_artist_song_query = insert_artist_song_query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(insert_artist_song_query, (line[0], line[9], int(line[8]), int(line[3]), float(line[5])))
        

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
query_artist_song = 'SELECT artist_name, song_name, song_length FROM artist_song WHERE session_id = 338 AND item_in_session = 4'
try:
    rows = session.execute(query_artist_song)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_name, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [11]:
# Testing question 1 
df = pd.read_csv("event_datafile_new.csv")
df[(df["sessionId"] == 338) & (df["itemInSession"] == 4)].head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
444,Faithless,Ava,F,4,Robinson,495.3073,free,"New Haven-Milford, CT",338,Music Matters (Mark Knight Dub),50


### Copy and paste the 3 above cells bellow to answer questions 2 and 3

In [12]:
# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
# for userid = 10, sessionid = 182
drop_artist_song_user_query = "DROP TABLE IF EXISTS artist_song_user;"

create_artist_song_user_query = "CREATE TABLE IF NOT EXISTS artist_song_user "
create_artist_song_user_query = create_artist_song_user_query + """
    (artist_name text, song_name text, user_first_name text, user_last_name text, item_in_session int, session_id int, user_id int, PRIMARY KEY ((user_id, session_id), item_in_session)) 
    WITH CLUSTERING ORDER BY (item_in_session ASC);
"""
try:
    session.execute(drop_artist_song_user_query)
    session.execute(create_artist_song_user_query)
except Exception as e:
    print(e)
                           

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insert_artist_song_user_query = "INSERT INTO artist_song_user (artist_name, song_name, user_first_name, user_last_name, item_in_session, session_id, user_id)"
        insert_artist_song_user_query = insert_artist_song_user_query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(insert_artist_song_user_query, (line[0], line[9], line[1], line[4], int(line[3]), int(line[8]), int(line[10])))
        

In [14]:
query_artist_song_user = 'SELECT artist_name, song_name, user_first_name, user_last_name FROM artist_song_user WHERE user_id = 10 AND session_id = 182'
try:
    rows = session.execute(query_artist_song_user)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_name, row.user_first_name, row.user_last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [15]:
# Testing question 2
df[(df["sessionId"] == 182) & (df["userId"] == 10)].head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
4704,Down To The Bone,Sylvie,F,0,Cruz,333.76608,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Keep On Keepin' On,10
4705,Three Drives,Sylvie,F,1,Cruz,411.63710,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Greece 2000,10
4706,Sebastien Tellier,Sylvie,F,2,Cruz,377.73016,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Kilometer,10
4707,Lonnie Gordon,Sylvie,F,3,Cruz,181.21098,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Catch You Baby (Steve Pitron & Max Sanna Radio...,10


In [16]:
# Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
drop_user_name_song_query = "DROP TABLE IF EXISTS user_name_song;"

create_user_name_song_query = "CREATE TABLE IF NOT EXISTS user_name_song "
create_user_name_song_query = create_user_name_song_query + """
    (user_first_name text, user_last_name text, song_name text, user_id int, PRIMARY KEY ((song_name), user_id)) 
"""
try:
    session.execute(drop_user_name_song_query)
    session.execute(create_user_name_song_query)
except Exception as e:
    print(e)
                    

In [17]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insert_user_name_song_query = "INSERT INTO user_name_song (user_first_name, user_last_name, song_name, user_id)"
        insert_user_name_song_query = insert_user_name_song_query + " VALUES (%s, %s, %s, %s)"
        session.execute(insert_user_name_song_query, (line[1], line[4], line[9], int(line[10])))
        

In [18]:
query_user_name_song = "SELECT user_first_name, user_last_name FROM user_name_song WHERE song_name = 'All Hands Against His Own'"
try:
    rows = session.execute(query_user_name_song)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_first_name, row.user_last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


In [19]:
# Testing question 3
df[(df["song"] == "All Hands Against His Own")].head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
2792,The Black Keys,Tegan,F,25,Levine,196.91057,paid,"Portland-South Portland, ME",611,All Hands Against His Own,80
5135,The Black Keys,Sara,F,31,Johnson,196.91057,paid,"Winston-Salem, NC",152,All Hands Against His Own,95
6298,The Black Keys,Jacqueline,F,50,Lynch,196.91057,paid,"Atlanta-Sandy Springs-Roswell, GA",559,All Hands Against His Own,29


### Drop the tables before closing out the sessions

In [20]:
try:
    session.execute(drop_artist_song_query)
    session.execute(drop_artist_song_user_query)
    session.execute(drop_user_name_song_query)

except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()